In [ ]:
def FDL(output_folder_name, num_days, seed_number, num_rounds, patients_included, patients_included_name, mins_before_predict_list, window_slide_list, epochs, batch_size, learning_rate,monitor):
    output_folder_results = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(patients_included_name)+'/results'
    os.makedirs(output_folder_results, exist_ok=True) 
    output_folder_models = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(patients_included_name)+'/models'
    os.makedirs(output_folder_models, exist_ok=True)
#################################### generate the local model #########################################################################################################  
    results_df = pd.DataFrame(columns=['Round', 'File', 'mins_before_predict', 'window_slide', 'RMSE_global', 'RMSE_local'])

    for round in range(num_rounds):
        print("round: "+str(round))
        RMSE_local = []
        RMSE_global = []
        local_weights_list = [] 
        for i in range(len(patients_included)):
            random.seed(seed_number) 
            np.random.seed(seed_number)
            tf.random.set_seed(seed_number)
            #######get X and y####################
            file = patients_included[i]
            directory = os.getcwd()

            file_path = os.path.join(directory, file)
            print(file_path)
            df = pd.read_csv(file_path)
            df = df[0:480*num_days] ###########use 5 days datat only

            df_training = df.iloc[:int(480*(num_days-2))]
            df_validation = df.iloc[int(480*(num_days-2)):int(480*(num_days-1))]
            df_testing = df.iloc[int(480*(num_days-1)):480*num_days]


            # Extract the input features and target variable
            X_training = df_training[['insulin', 'CHO', 'CGM']]
            y_training = df_training['CGM']

            X_validation = df_validation[['insulin', 'CHO', 'CGM']]
            y_validation = df_validation['CGM']

            X_testing = df_testing[['insulin', 'CHO', 'CGM']]
            y_testing = df_testing['CGM']
            # Convert the DataFrame to numpy arrays
            X_training= X_training.to_numpy()
            y_training = y_training.to_numpy()

            X_validation= X_validation.to_numpy()
            y_validation = y_validation.to_numpy()

            X_testing= X_testing.to_numpy()
            y_testing = y_testing.to_numpy()
            for mins_before_predict in mins_before_predict_list:
                print("mins_before_predict: "+str(mins_before_predict))
                timesteps = int(mins_before_predict/3)
                for window_slide in window_slide_list:
                    print("window slide: "+str(window_slide))
                    RMSE = []
                    

                    # Create input sequences and corresponding target values
                    X_sequences_training = []
                    y_targets_training = []

                    X_sequences_validation = []
                    y_targets_validation = []

                    X_sequences_testing = []
                    y_targets_testing = []

                    for i in range(0, len(X_training) - timesteps-window_slide):
                        X_sequences_training.append(X_training[i:i+window_slide])
                        y_targets_training.append(y_training[i+timesteps+window_slide-1])

                    # Convert sequences and targets to numpy arrays
                    X_sequences_training = np.array(X_sequences_training)
                    y_targets_training = np.array(y_targets_training)

                    for i in range(0, len(X_validation) - timesteps-window_slide):
                        X_sequences_validation.append(X_validation[i:i+window_slide])
                        y_targets_validation.append(y_validation[i+timesteps+window_slide-1])

                    # Convert sequences and targets to numpy arrays
                    X_sequences_validation = np.array(X_sequences_validation)
                    y_targets_validation = np.array(y_targets_validation)


                    for i in range(0, len(X_testing) - timesteps-window_slide):
                        X_sequences_testing.append(X_testing[i:i+window_slide])
                        y_targets_testing.append(y_testing[i+timesteps+window_slide-1])

                    # Convert sequences and targets to numpy arrays
                    X_sequences_testing = np.array(X_sequences_testing)
                    y_targets_testing = np.array(y_targets_testing)
                    
                    if round == 0:
                        filepath_global_model = os.path.join(output_folder_models, 'best_model_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+".h5")
                    else:
                        filepath_global_model = os.path.join(output_folder_models, 'global_round_'+str(round-1)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5')



                    # Define the local client model
                    global_model = load_model(filepath_global_model)
                    y_pred_global = global_model.predict(X_sequences_testing)
                    RMSE_values_global = np.sqrt(mean_squared_error(y_targets_testing, y_pred_global))
                    print('RMSE value of the global model: %.4f' %RMSE_values_global)


                    local_model = load_model(filepath_global_model)
                    # Define a ModelCheckpoint callback to save the best model based on MSE
                    file_name = 'FDL_'+str(num_days)+'_'+str(file)+'_round_'+str(round)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'
                    filepath = os.path.join(output_folder_models, file_name)


                    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True, save_weights_only=False, monitor=monitor)
                    early_stopper = EarlyStopping(monitor=monitor, mode='auto', patience=5)
                    # Compile the network 
                    optimizer = Adam(learning_rate=learning_rate)
                    local_model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

                    # Iterate over the federated learning rounds
                    # Train the local client model on its data
                    # Train the model with validation data and callbacks
                    history = local_model.fit(X_sequences_training, y_targets_training, batch_size=batch_size, epochs = epochs, verbose=1, validation_data=(X_sequences_validation, y_targets_validation), callbacks=[checkpointer, early_stopper])

                    # Load the best model based on MSE
                    local_model = load_model(filepath)

                    local_weights = local_model.get_weights()
                    local_weights_list.append(local_weights)

                    y_pred = local_model.predict(X_sequences_testing)
                    RMSE_values_local = np.sqrt(mean_squared_error(y_targets_testing, y_pred))
                    print('RMSE value of local model: %.4f' %RMSE_values_local)



                    x_test_values = range(len(X_sequences_testing))
                    #########all rows######################
                    fig, ax = plt.subplots(figsize=(16, 10))
                    ax.plot(x_test_values, y_pred, '-', color = "dodgerblue",label = "predicted",lw=6)
                    ax.plot(x_test_values, y_targets_testing, '-',color = "darkorange",label = "real",lw=6)

                    plt.rcParams['font.family'] = 'Times New Roman'
                    plt.rcParams.update({'font.size': 36})
                    plt.rcParams.update({'axes.labelweight': "bold"})

                    
                    ax.set_xlabel('hour', fontsize=36)
                    ax.set_ylabel('CGM (mg/dl)', fontsize=36)
                    ax.set_xlim(0, 480)
                    ax.set_xticks(range(0, 481, 60))
                    ax.set_xticklabels(range(0, 25, 3), fontsize=36)

                    ax.set_ylim(0, max(y_pred)*1.2)  # Set the lower and upper limits

                    plt.legend(loc='upper center',  fontsize=36, ncol=2)
                    plt.xticks(fontsize=36)
                    plt.yticks(fontsize=36)
                    # Show the plot
                    plt.show()

                    RMSE_local.append(RMSE_values_local)  # Append RMSE value to the list
                    RMSE_global.append(RMSE_values_global) 
                    new_global_weight = [np.mean(w, axis=0) for w in zip(*local_weights_list)]
                    global_model.set_weights(new_global_weight)
                    file_name_global = 'global_round_'+str(round)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'
                    global_model.save(os.path.join(output_folder_models, file_name_global))

                    # Define the path to the saved model
                    model_path = os.path.join(output_folder_models, file_name_global)

                    # Append the results to the DataFrame
                    new_data = {'Round': str(round), 'File': file, 'mins_before_predict': mins_before_predict,'window_slide': window_slide, 'RMSE_global': RMSE_values_global, 'RMSE_local': RMSE_values_local}
                    new_df = pd.DataFrame([new_data])
                    # Concatenate the existing results_df DataFrame with the new DataFrame
                    results_df = pd.concat([results_df, new_df], ignore_index=True)

    # Save the DataFrame to an Excel file
    output_filename = os.path.join(output_folder_results, str(function_name)+'.xlsx')
    results_df.to_excel(output_filename, index=False)